In [1]:
from pyGROMACS import GROMACS_setup

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



## Solvation free energy

This notebook setup and analyse simulations to compute the solvation free energy of a provided solute in the mixture

<p align="center">
    <img src="pipeline_solvation.png" alt="Pipeline Image" width="1000"/>
</p>

1) [Optional] Add Guest molecule and equilibrate system
2) [Optional] Optimize the intermediates for decoupling
3) Setup decoupling simulations for each solute for each lambda
4) Analyse the free solvation energy

In [2]:
gromacs_setup = GROMACS_setup( system_setup = "input/system.yaml", 
                               simulation_default = "input/defaults.yaml",
                               simulation_ensemble = "input/ensemble.yaml",
                               submission_command = "qsub"
                             )

### 1) [Optional] Add Guest molecule and equilibrate system

In [7]:
# Define the simulation folder
simulation_folder = "solvation_free_energy"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "nvt", "npt_equilibration", "npt_production" ]

# Define the simulation time per ensemble in nano seconds (for em provide number of iterations)
simulation_times = [ 10000, 1.0, 1.0, 1.0 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# Therefore, provide one initial structure for each temperature & pressure state.
# If none are specified, gmx-insert will create them.
initial_systems = []

# Provide kwargs that should be passed into the mdp template directly
mdp_kwargs = {}

# Define number of copies
copies = 2

# Define if the inital system should build locally or with the cluster
on_cluster = False

# Define the solutes (names should match the residue)
solutes = [ "CO2" ]


for solute in solutes:

    # Increase the molecule number of the solute in the system by one
    for mol_dict in gromacs_setup.system_setup["molecules"]:
        if mol_dict["name"] == solute:
            mol_dict["number"] +=1
            break


    gromacs_setup.prepare_simulation( folder_name = f"{simulation_folder}/{solute}/equilibration", ensembles = ensembles, 
                                      simulation_times = simulation_times, initial_systems = initial_systems, 
                                      mdp_kwargs = mdp_kwargs, copies = copies, on_cluster = on_cluster 
                                    )

    # Decrease it again, that next solute has correct system
    mol_dict["number"] -=1

State: T = 313.1 K, P = 1.0 bar

Building system based on provided molecule numbers and coordinate files!


Build system locally! Wait until it is finished.

Build successful



In [8]:
# Submit the simulations
gromacs_setup.submit_simulation()


Submitting simulations at Temperature = 313 K, Pressure = 1 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/equilibration/temp_313.1_pres_1.0/copy_0/job_313.sh
11444842


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/equilibration/temp_313.1_pres_1.0/copy_1/job_313.sh
11444843


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/equilibration/temp_313.1_pres_1.0/copy_2/job_313.sh
11444844




### 2) [Optional] Optimize the intermediates for decoupling

### 3) Setup decoupling simulations for each solute for each lambda

In [3]:
# Define the simulation folder
simulation_folder = "solvation_free_energy"

# Define ensembles, the meaning of each ensemble is defined in the simulation_ensemble yaml
ensembles = [ "em", "nvt", "npt_equilibration", "npt_production" ]

# Define simulation time per ensemble in nanoseconds (for "em" provide number of optimization steps)
simulation_times = [ 10000, 2.0, 2.0, 10.0 ]

# Define the number of copies for each system
copies = 0

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

# Solutes
solutes = [ "CO2" ]

# Define lambdas that should be used (for each solute individual)
combined_lambdas = [ [0.00, 0.25, 0.50, 0.75, 1.00, 1.05, 1.10, 1.20, 1.30, 1.40, 1.50, 1.60, 1.65, 1.70, 1.75, 1.80, 1.85, 1.90, 1.95, 2.00 ] ]

# Ensure that all jobs for all solutes are submitted at once
job_files = [ [] for _ in gromacs_setup.system_setup["temperature"] ]

for i,solute in enumerate( solutes ):

    # Define initial systems for each temperature & pressure state, based on the equilibration done before
    initial_systems = [ f"/home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/{solute}/equilibration/temp_313.1_pres_1.0/copy_1/03_npt_production/npt_production.gro" ]

    # Increase the molecule number of the solute in the system by one (important for toplogy file and if initial system is not provided)
    for mol_dict in gromacs_setup.system_setup["molecules"]:
        if mol_dict["name"] == solute:
            mol_dict["number"] +=1
            break

    gromacs_setup.prepare_free_energy_simulation( folder_name = simulation_folder, simulation_free_energy = "input/free_energy.yaml", solute = solute, 
                                                  combined_lambdas = combined_lambdas[i], initial_systems = initial_systems, ensembles = ensembles,
                                                  copies = copies, simulation_times = simulation_times, off_set = off_set  )

    # Decrease it again, that next solute has correct system
    mol_dict["number"] -=1
    
    for j, files in enumerate( gromacs_setup.job_files ):
      job_files[j].extend( files )

gromacs_setup.job_files = job_files

State: T = 313.1 K, P = 1.0 bar

System already build and initial configuration is provided at:
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/equilibration/temp_313.1_pres_1.0/copy_1/03_npt_production/npt_production.gro

Checkpoint file (.cpt) is provided in the same folder.



In [4]:
# Submit the simulations
gromacs_setup.submit_simulation()


Submitting simulations at Temperature = 313.1 K, Pressure = 1.0 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_0/job_313_lambda_0.sh
11444861


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_1/job_313_lambda_1.sh
11444862


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_2/job_313_lambda_2.sh
11444863


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_3/job_313_lambda_3.sh
11444864


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambd

## Data analysis

This generic tool allows to extract properties using GROMACS commands for an specified simulation folder and ensemble. This will average the properties over each copy and safe the produced data in each temperature & pressure folder as json file


### Extract and analysis of solvation free energies

In [5]:
# Extract solvation free energy results 

# Define analysis folder
analysis_folder = "solvation_free_energy"

# Define analysis ensemble
ensemble = "03_npt_production"

# Solutes
solutes  = [ "CO2" ] 

# Free energy method (MBAR, BAR, TI, TI_spline)
method = "MBAR"

# Decorrelate free energy data (using alchemlyb tools)
decorrelate = False

# Define if coupling (True) or decoupling (False) is performed
coupling = False

for solute in solutes:
    gromacs_setup.analysis_free_energy( analysis_folder = analysis_folder, ensemble = ensemble, solute = solute,
                                        method = method, fraction = 0.25, decorrelate = decorrelate, 
                                        visualize = True, coupling = coupling )


Extract solvation free energy results for solute: CO2

Temperature: 313.15 K, Pressure: 1.0 bar
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_0/03_npt_production/npt_production.xvg
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_1/03_npt_production/npt_production.xvg
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_2/03_npt_production/npt_production.xvg
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_3/03_npt_production/npt_production.xvg
   /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/solvation/co2_water/solvation_free_energy/CO2/temp_313.1_pres_1.0/copy_0/lambda_4/03_npt_pro


******* JAX 64-bit mode is now on! *******
*     JAX is now set to 64-bit mode!     *
*   This MAY cause problems with other   *
*      uses of JAX in the same code.     *
******************************************




Following combined lambda states were analysed with the 'MBAR' method:
   0.0, 0.25, 0.5, 0.75, 1.0, 1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1.9, 1.95, 2.0

Averaged values over all copies:

                 property      mean       std    unit
0  solvation_free_energy  2.113476  0.027134  kJ/mol 

